# Named Tensor Notation

## Introduction

**This is a translation of the [Named Tensor Notation (Chiang, Rush, Barak 2021)](https://namedtensor.github.io/) example from the [funsor](https://funsor.pyro.ai/) library to `effectful`. Much of the expository text is taken directly from the [original](https://github.com/pyro-ppl/funsor/blob/master/tutorials/named_tensor_notation_i.ipynb).**

The mathematical notation with *named axes* introduced in [Named Tensor Notation (Chiang, Rush, Barak 2021)](https://namedtensor.github.io/) improves the readability of mathematical formulas involving multidimensional arrays. This includes tensor operations such as elementwise operations, reductions, contractions, renaming, indexing, and broadcasting. Part 1 covers examples from [2 Informal Overview](https://namedtensor.github.io/#sec:overview), [3.4.2 Advanced Indexing](https://namedtensor.github.io/#sec:examples), and [5 Formal Definitions](https://namedtensor.github.io/#sec:definitions).

In [75]:
import functools

import torch
from torch import tensor

from effectful.ops.core import evaluate, ctxof
from effectful.ops.handler import handler
from effectful.internals.sugar import gensym, torch_getitem, sizesof
from effectful.indexed.ops import Indexable, to_tensor


def subst(term, substs):
    with handler(
        {k: functools.partial(lambda vv: vv, v) for (k, v) in substs.items()},
    ):
        return evaluate(term)


def reduce(indexes, indexed_tensor, reducer):
    """Reduce an indexed tensor along one or more named dimensions.

    Args:
    - indexes: Names of dimensions to reduce.
    - indexed_tensor: The tensor to reduce.
    - reducer: A reduction function like `torch.sum`. Must take `tensor`, `dim`, and `keepdim` arguments.

    Returns: A new indexed tensor with the specified dimensions reduced.

    Example:
    >>> width, height = gensym(int, name='width'), gensym(int, name='height')
    >>> t = indexed(torch.ones(2, 3))[width(), height()]
    >>> reduce([width], t, "sum")
    indexed(tensor([2., 2., 2.]))[height()]
    """
    fvars = ctxof(indexed_tensor)
    indexes = [i for i in indexes if i in fvars]

    # convert indexed dimensions to positional and flatten all new positional dims
    t = to_tensor(indexed_tensor, indexes)
    t_flat = torch.flatten(t, 0, len(indexes) - 1)

    # reduce dim 0 into the first index of dim 0, then return reduction
    return reducer(t_flat, 0, keepdim=True)[0]


def gensyms(*names, type_=int):
    return tuple(gensym(int, name=n) for n in names)

## Named Tensors

Each tensor axis is given a name:

$$
\begin{aligned}
  A &\in \mathbb{R}^{\mathsf{\vphantom{fg}height}[3] \times \mathsf{\vphantom{fg}width}[3]} = \mathbb{R}^{\mathsf{\vphantom{fg}width}[3] \times \mathsf{\vphantom{fg}height}[3]} \\
  A &= \mathsf{\vphantom{fg}height}
  \begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}\\\begin{bmatrix}
    3 & 1 & 4 \\
    1 & 5 & 9 \\
    2 & 6 & 5
  \end{bmatrix}\end{array} =
  \mathsf{\vphantom{fg}width}
  \begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}height}\\\begin{bmatrix}
    3 & 1 & 2 \\
    1 & 5 & 6 \\
    4 & 9 & 5
  \end{bmatrix}\end{array}.
\end{aligned}
$$

In [76]:
height, width = gensym(int, name="height"), gensym(int, name="width")
t = tensor([[3, 1, 4], [1, 5, 9], [2, 6, 5]])
A = Indexable(tensor([[3, 1, 4], [1, 5, 9], [2, 6, 5]]))[height(), width()]
A

Indexable(tensor([[3, 1, 4],
                  [1, 5, 9],
                  [2, 6, 5]]))[height(), width()]

Access elements of $A$ using named indices:

$$
A_{\mathsf{\vphantom{fg}height}(1), \mathsf{\vphantom{fg}width}(3)} = A_{\mathsf{\vphantom{fg}width}(3), \mathsf{\vphantom{fg}height}(1)} = 4
$$

In [77]:
subst(A, {height: 0, width: 2})

tensor(4)

Partial indexing:

$$
\begin{aligned}
A_{\mathsf{\vphantom{fg}height}(1)} &= \begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}\\
\begin{bmatrix}
  3 & 1 & 4
\end{bmatrix}\end{array}
&
A_{\mathsf{\vphantom{fg}width}(3)} &= \begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}height}\\
\begin{bmatrix}
  4 & 9 & 5
\end{bmatrix}\end{array}.
\end{aligned}
$$

In [78]:
subst(A, {height: 0})

Indexable(tensor([3, 1, 4]))[width()]

In [79]:
subst(A, {width: 2})

Indexable(tensor([4, 9, 5]))[height()]

## Named Tensor Operations

### Elementwise Operations and Broadcasting

Elementwise operations:

$$
\frac1{1+\exp(-A)} = \mathsf{\vphantom{fg}height}
\begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}\\
\begin{bmatrix}
  \frac 1{1+\exp(-3)} & \frac 1{1+\exp(-1)} & \frac 1{1+\exp(-4)} \\[1ex]
  \frac 1{1+\exp(-1)} & \frac 1{1+\exp(-5)} & \frac 1{1+\exp(-9)} \\[1ex]
  \frac 1{1+\exp(-2)} & \frac 1{1+\exp(-6)} & \frac 1{1+\exp(-5)}
\end{bmatrix}\end{array}.
$$

In [80]:
1 / (1 + (-A).exp())

Indexable(tensor([[0.9526, 0.7311, 0.9820],
                  [0.7311, 0.9933, 0.9999],
                  [0.8808, 0.9975, 0.9933]]))[height(), width()]

Tensors with different shapes are automatically broadcasted against each other before an operation is applied. Let

$$
\begin{aligned}
  x &\in \mathbb{R}^{\mathsf{\vphantom{fg}height}[3]} & y &\in \mathbb{R}^{\mathsf{\vphantom{fg}width}[3]} \\
  x &= \mathsf{\vphantom{fg}height}
  \begin{array}[b]{@{}c@{}}\\
  \begin{bmatrix}
    2 \\ 7 \\ 1
  \end{bmatrix}\end{array} & 
  y &= 
  \begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}\\\begin{bmatrix}
    1 & 4 & 1
  \end{bmatrix}\end{array}.
\end{aligned}
$$

In [81]:
x = Indexable(tensor([2, 7, 1]))[height()]
y = Indexable(tensor([1, 4, 1]))[width()]

Binary addition operation:

$$
\begin{aligned}
A + x &= \mathsf{\vphantom{fg}height}
\begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}\\\begin{bmatrix}
  3+2 & 1+2 & 4+2 \\
  1+7 & 5+7 & 9+7 \\
  2+1 & 6+1 & 5+1
\end{bmatrix}\end{array} &
A + y &= \mathsf{\vphantom{fg}height}
\begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}\\\begin{bmatrix}
  3+1 & 1+4 & 4+1 \\
  1+1 & 5+4 & 9+1 \\
  2+1 & 6+4 & 5+1
\end{bmatrix}\end{array}.
\end{aligned}
$$

In [82]:
A + x

Indexable(tensor([[ 5,  3,  6],
                  [ 8, 12, 16],
                  [ 3,  7,  6]]))[height(), width()]

In [83]:
A + y

Indexable(tensor([[ 4,  5,  5],
                  [ 2,  9, 10],
                  [ 3, 10,  6]]))[height(), width()]

Binary multiplication operation:

$$
A \odot x = \mathsf{\vphantom{fg}height}
\begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}\\\begin{bmatrix}
  3\cdot2 & 1\cdot2 & 4\cdot2 \\
  1\cdot7 & 5\cdot7 & 9\cdot7 \\
  2\cdot1 & 6\cdot1 & 5\cdot1
\end{bmatrix}\end{array}
$$

In [84]:
A * x

Indexable(tensor([[ 6,  2,  8],
                  [ 7, 35, 63],
                  [ 2,  6,  5]]))[height(), width()]

Binary maximum operation:

$$
\max(A, y) = \mathsf{\vphantom{fg}height}
\begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}\\\begin{bmatrix}
  \max(3, 1) & \max(1, 4) & \max(4, 1) \\
  \max(1, 1) & \max(5, 4) & \max(9, 1) \\
  \max(2, 1) & \max(6, 4) & \max(5, 1)
\end{bmatrix}\end{array}.
$$

In [85]:
torch.max(A, y)

Indexable(tensor([[3, 4, 4],
                  [1, 5, 9],
                  [2, 6, 5]]))[height(), width()]

### Reductions

Named axes can be reduced over by calling the `.reduce` method and specifying the [reduction operator](https://en.wikipedia.org/wiki/Reduction_Operator) and names of reduced axes. Note that reduction is defined only for operators that are associative and commutative.

$$
\sum\limits_{\substack{\mathsf{\vphantom{fg}height}}} A = \sum_i A_{\mathsf{\vphantom{fg}height}(i)} = \begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}\\
\begin{bmatrix}
  3+1+2 & 1+5+6 & 4+9+5
\end{bmatrix}\end{array}.
$$

In [86]:
reduce([height], A, torch.sum)

Indexable(tensor([ 6, 12, 18]))[width()]

$$
\sum\limits_{\substack{\mathsf{\vphantom{fg}width}}} A = \sum_j A_{\mathsf{\vphantom{fg}width}(j)} = \begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}height}\\
\begin{bmatrix}
  3+1+4 & 1+5+9 & 2+6+5
\end{bmatrix}\end{array}.
$$

In [87]:
reduce([width], A, torch.sum)

Indexable(tensor([ 8, 15, 13]))[height()]

Reduction over multiple axes:

$$
\sum\limits_{\substack{\mathsf{\vphantom{fg}height}\\
 \mathsf{\vphantom{fg}width}}} A = \sum_i \sum_j A_{\mathsf{\vphantom{fg}height}(i),\mathsf{\vphantom{fg}width}(j)} = 3+1+4+1+5+9+2+6+5.
 $$

In [88]:
reduce([height, width], A, torch.sum)

tensor(36)

Multiplication reduction:

$$
\prod\limits_{\substack{\mathsf{\vphantom{fg}height}}} A = \prod_i A_{\mathsf{\vphantom{fg}height}(i)} = \begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}\\
\begin{bmatrix}
  3\cdot1\cdot2 & 1\cdot5\cdot6 & 4\cdot9\cdot5
\end{bmatrix}\end{array}.
$$

In [89]:
reduce([height], A, torch.prod)

Indexable(tensor([  6,  30, 180]))[width()]

Max reduction:

$$
\max\limits_{\substack{\mathsf{\vphantom{fg}height}}} A = \max \{A_{\mathsf{\vphantom{fg}height}(i)} \mid 1 \leq i \leq n\} = \begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}\\
\begin{bmatrix}
  \max(3, 1, 2) & \max(1, 5, 6) & \max(4, 9, 5)
\end{bmatrix}\end{array}.
$$

In [90]:
reduce([height], A, torch.amax)

Indexable(tensor([3, 6, 9]))[width()]

### Contraction

Contraction operation can be written as elementwise multiplication followed by summation over an axis:

$$
A \mathbin{\underset{\substack{\mathsf{\vphantom{fg}width}}}{\vphantom{fg}\odot}} y = \sum_j A_{\mathsf{\vphantom{fg}width}(j)} \, y_{\mathsf{\vphantom{fg}width}(j)} = \mathsf{\vphantom{fg}height}
\begin{array}[b]{@{}c@{}}\\\begin{bmatrix}
  3\cdot 1 + 1\cdot 4 + 4\cdot 1 \\
  1\cdot 1 + 5\cdot 4 + 9\cdot 1 \\
  2\cdot 1 + 6\cdot 4 + 5\cdot 1
\end{bmatrix}\end{array}.
$$

In [91]:
reduce([width], A * y, torch.sum)

Indexable(tensor([11, 30, 31]))[height()]

Some other operations from linear algebra:

$$
x \mathbin{\underset{\substack{\mathsf{\vphantom{fg}height}}}{\vphantom{fg}\odot}} x = \sum_i x_{\mathsf{\vphantom{fg}height}(i)} \, x_{\mathsf{\vphantom{fg}height}(i)} \qquad \text{inner product}
$$

In [92]:
reduce([height], x * x, torch.sum)

tensor(54)

$$
[x \odot y]_{\mathsf{\vphantom{fg}height}(i), \mathsf{\vphantom{fg}width}(j)} = x_{\mathsf{\vphantom{fg}height}(i)} \, y_{\mathsf{\vphantom{fg}width}(j)} \qquad \text{outer product}
$$

In [93]:
x * y

Indexable(tensor([[ 2,  8,  2],
                  [ 7, 28,  7],
                  [ 1,  4,  1]]))[height(), width()]

$$
A \mathbin{\underset{\substack{\mathsf{\vphantom{fg}width}}}{\vphantom{fg}\odot}} y = \sum_i A_{\mathsf{\vphantom{fg}width}(i)} \, y_{\mathsf{\vphantom{fg}width}(i)} \qquad \text{matrix-vector product}
$$

In [94]:
reduce([width], A * y, torch.sum)

Indexable(tensor([11, 30, 31]))[height()]

$$
x \mathbin{\underset{\substack{\mathsf{\vphantom{fg}height}}}{\vphantom{fg}\odot}} A = \sum_i x_{\mathsf{\vphantom{fg}height}(i)} \, A_{\mathsf{\vphantom{fg}height}(i)} \qquad \text{vector-matrix product} \\
$$

In [95]:
reduce([height], x * A, torch.sum)

Indexable(tensor([15, 43, 76]))[width()]

$$
A \mathbin{\underset{\substack{\mathsf{\vphantom{fg}width}}}{\vphantom{fg}\odot}} B = \sum_i A_{\mathsf{\vphantom{fg}width}(i)} \odot B_{\mathsf{\vphantom{fg}width}(i)} \qquad \text{matrix-matrix product}~(B \in \mathbb{R}^{\mathsf{\vphantom{fg}width}\times \mathsf{\vphantom{fg}width2}})
$$

In [96]:
width2 = gensym(int, name="width2")
B = Indexable(
    tensor([[3, 2, 5], [5, 4, 0], [8, 3, 6]]),
)[width(), width2()]

reduce([width], A * B, torch.sum)

Indexable(tensor([[ 46,  22,  39],
                  [100,  49,  59],
                  [ 76,  43,  40]]))[height(), width2()]

Contraction can be generalized to other binary and reduction operations:

$$
\max_{\mathsf{\vphantom{fg}width}} (A + y) = \mathsf{\vphantom{fg}height}
\begin{array}[b]{@{}c@{}}\\\begin{bmatrix}
  \max(3+1, 1+4, 4+1) \\
  \max(1+1, 5+4, 9+1) \\
  \max(2+1, 6+4, 5+1)
\end{bmatrix}\end{array}.
$$

In [97]:
reduce([width], A + y, torch.amax)

Indexable(tensor([ 5, 10, 10]))[height()]

### Renaming and Reshaping

Renaming named dimensions is simple:

$$
A_{\mathsf{\vphantom{fg}height}\rightarrow\mathsf{\vphantom{fg}height2}} = \mathsf{\vphantom{fg}height2}
\begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}
\\\begin{bmatrix}
  3 & 1 & 4 \\
  1 & 5 & 9 \\
  2 & 6 & 5 \\
\end{bmatrix}\end{array}.
$$

In [98]:
height2 = gensym(int, name="height2")
subst(A, {height: height2()})

Indexable(tensor([[3, 1, 4],
                  [1, 5, 9],
                  [2, 6, 5]]))[height2(), width()]

$$
A_{(\mathsf{\vphantom{fg}height},\mathsf{\vphantom{fg}width})\rightarrow\mathsf{\vphantom{fg}layer}} = \begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}layer}\\
\begin{bmatrix}
    3 & 1 & 4 & 1 & 5 & 9 & 2 & 6 & 5
\end{bmatrix}\end{array}
$$

In [99]:
layer = gensym(int, name="layer")
A_layer = subst(A, {height: layer() // 3, width: layer() % 3})
print(subst(A_layer, {layer: 2}))

tensor(4)


$$
A_{\mathsf{\vphantom{fg}layer}\rightarrow(\mathsf{\vphantom{fg}height},\mathsf{\vphantom{fg}width})} = \mathsf{\vphantom{fg}height}
\begin{array}[b]{@{}c@{}}\mathsf{\vphantom{fg}width}
\\\begin{bmatrix}
  3 & 1 & 4 \\
  1 & 5 & 9 \\
  2 & 6 & 5 \\
\end{bmatrix}\end{array}.
$$

In [100]:
print(subst(A_layer, {layer: height() * 3 + width() % 3}))

_torch_op(tensor([[3, 1, 4],
        [1, 5, 9],
        [2, 6, 5]]), [floordiv(add(mul(height(), 3), mod(width(), 3)), 3), mod(add(mul(height(), 3), mod(width(), 3)), 3)])


## Advanced Indexing

All of advanced indexing can be achieved through name substitutions.

$$
\mathop{\underset{\substack{\mathsf{\vphantom{fg}ax}}}{\vphantom{fg}\mathrm{index}}} \colon \mathbb{R}^{\mathsf{\vphantom{fg}ax}[n]} \times [n] \rightarrow \mathbb{R}\\
\mathop{\underset{\substack{\mathsf{\vphantom{fg}ax}}}{\vphantom{fg}\mathrm{index}}}(A, i) = A_{\mathsf{\vphantom{fg}ax}(i)}.
$$

$$
\begin{aligned}
  E &\in \mathbb{R}^{\mathsf{\vphantom{fg}vocab}[n] \times \mathsf{\vphantom{fg}emb}} \\
  i &\in [n] \\
  I &\in [n]^{\mathsf{\vphantom{fg}seq}} \\
  P &\in \mathbb{R}^{\mathsf{\vphantom{fg}seq}\times \mathsf{\vphantom{fg}vocab}[n]}
\end{aligned}
$$

Partial indexing $\mathop{\underset{\substack{\mathsf{\vphantom{fg}vocab}}}{\vphantom{fg}\mathrm{index}}}(E,i)$:

In [101]:
vocab, emb = gensym(int, name="vocab"), gensym(int, name="emb")
E = Indexable(
    tensor([[2, 1, 5], [3, 4, 2], [1, 3, 7], [1, 4, 3], [5, 9, 2]]),
)[vocab(), emb()]

subst(E, {vocab: 2})

Indexable(tensor([1, 3, 7]))[emb()]

Integer array indexing $\mathop{\underset{\substack{\mathsf{\vphantom{fg}vocab}}}{\vphantom{fg}\mathrm{index}}}(E,I)$:

In [102]:
seq = gensym(int, name="seq")
I = Indexable(tensor([3, 2, 4, 0]))[seq()]

subst(E, {vocab: I})

Indexable(tensor([[1, 4, 3],
                  [1, 3, 7],
                  [5, 9, 2],
                  [2, 1, 5]]))[seq(), emb()]

Gather operation $\mathop{\underset{\substack{\mathsf{\vphantom{fg}vocab}}}{\vphantom{fg}\mathrm{index}}}(P,I)$:

In [103]:
P = Indexable(
    tensor([[6, 2, 4, 2], [8, 2, 1, 3], [5, 5, 7, 0], [1, 3, 8, 2], [5, 9, 2, 3]]),
)[vocab(), seq()]

subst(P, {vocab: I})

Indexable(tensor([1, 5, 2, 2]))[seq()]

Indexing with two integer arrays:

$$
\begin{aligned}
  |\mathsf{\vphantom{fg}seq}| &= m \\
  I_1 &= [m]^\mathsf{\vphantom{fg}subseq}\\
  I_2 &= [n]^\mathsf{\vphantom{fg}subseq}\\
  S &= \mathop{\underset{\substack{\mathsf{\vphantom{fg}vocab}}}{\vphantom{fg}\mathrm{index}}}(\mathop{\underset{\substack{\mathsf{\vphantom{fg}seq}}}{\vphantom{fg}\mathrm{index}}}(P, I_1), I_2) \in \mathbb{R}^{\mathsf{\vphantom{fg}subseq}} \\
  S_{\mathsf{\vphantom{fg}subseq}(i)} &= P_{\mathsf{\vphantom{fg}seq}(I_{\mathsf{\vphantom{fg}subseq}(i)}), \mathsf{\vphantom{fg}vocab}(I_{\mathsf{\vphantom{fg}subseq}(i)})}.
\end{aligned}
$$

In [104]:
subseq = gensym(int, name="subseq")
I1 = Indexable(tensor([1, 2, 0]))[subseq()]
I2 = Indexable(tensor([3, 0, 4]))[subseq()]

subst(P, {seq: I1, vocab: I2})

Indexable(tensor([3, 4, 5]))[subseq()]

## Constructing Neural Networks

In [105]:
"""
from collections import OrderedDict
import functools

import math
import torch
from torch.distributions import constraints

%env FUNSOR_TYPECHECK=1
import funsor
from funsor.terms import Funsor, Variable, Number, Lambda, Slice
from funsor.tensor import Tensor
from funsor.domains import Array, Bint, Real, Reals
from funsor.factory import Bound, Fresh, Has, Value, make_funsor, to_funsor
import funsor.ops as ops
from funsor.cnf import Contraction
from funsor.testing import random_tensor
from funsor.interpretations import reflect, memoize
import funsor.torch.distributions as dist

funsor.set_backend("torch")
torch.set_default_dtype(torch.float32)
"""

import functools

import torch
from torch import tensor

from effectful.ops.core import evaluate, as_term, Operation
from effectful.ops.handler import handler
from effectful.internals.sugar import gensym, torch_getitem, Bound
from effectful.indexed.ops import Indexable, to_tensor


def reduce(indexes, indexed_tensor, reducer):
    """Reduce an indexed tensor along one or more named dimensions.

    Args:
    - indexes: Names of dimensions to reduce.
    - indexed_tensor: The tensor to reduce.
    - reducer: A reduction function like `torch.sum`. Must take `tensor`, `dim`, and `keepdim` arguments.

    Returns: A new indexed tensor with the specified dimensions reduced.

    Example:
    >>> width, height = gensym(int, name='width'), gensym(int, name='height')
    >>> t = indexed(torch.ones(2, 3))[width(), height()]
    >>> reduce([width], t, "sum")
    indexed(tensor([2., 2., 2.]))[height()]
    """
    num_positional = len(indexed_tensor.shape)
    # convert indexed dimensions to positional and flatten all new positional dims
    t = to_tensor(indexed_tensor, indexes)
    new_positional = len(t.shape) - num_positional
    t_flat = torch.flatten(t, 0, new_positional - 1)

    # reduce dim 0 into the first index of dim 0, then return reduction
    return reducer(t_flat, 0, keepdim=True)[0]

### Feedforward

\begin{aligned}
  X^0 &\in \mathbb{R}^{\mathsf{\vphantom{fg}input}} \\
  X^1 &= \sigma(W^1 \mathbin{\underset{\substack{\mathsf{\vphantom{fg}input}}}{\vphantom{fg}\odot}} X^0 + b^1) & W^1 &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}_1 \times \mathsf{\vphantom{fg}input}} & b^1 &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}_1} \\
  X^2 &= \sigma(W^2 \mathbin{\underset{\substack{\mathsf{\vphantom{fg}hidden}_1}}{\vphantom{fg}\odot}} X^1 + b^2) & W^2 &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}_2 \times \mathsf{\vphantom{fg}hidden}_1} & b^2 &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}_2} \\
  X^3 &= \sigma(W^3 \mathbin{\underset{\substack{\mathsf{\vphantom{fg}hidden}_2}}{\vphantom{fg}\odot}} X^2 + b^3) & W^3 &\in \mathbb{R}^{\mathsf{\vphantom{fg}out}\times \mathsf{\vphantom{fg}hidden}_2} & b^3 &\in \mathbb{R}^{\mathsf{\vphantom{fg}out}}
\end{aligned}

$$
\begin{aligned}
x &\in \mathbb{R}^{\mathsf{\vphantom{fg}layer}[n_0]} \\
W^l &\in \mathbb{R}^{\mathsf{\vphantom{fg}layer^2}[n_l] \times \mathsf{\vphantom{fg}layer}[n_{l-1}]} \\
  b^l &\in \mathbb{R}^{\mathsf{\vphantom{fg}layer^2}[n_l]} \\
  \text{FullConn}^l(x) &= \sigma\left(W^l \mathbin{\underset{\substack{\mathsf{\vphantom{fg}layer}}}{\vphantom{fg}\odot}} x + b^l\right)_{\mathsf{\vphantom{fg}layer^2}\rightarrow\mathsf{\vphantom{fg}layer}}
\end{aligned}
$$

In [106]:
@Operation
def FullConn(x: torch.Tensor, W: torch.Tensor, b: torch.Tensor, layer: Bound):
    return reduce([layer], torch.sigmoid(torch.mul(W, x)), torch.sum) + b

In [107]:
input_size = 100
output_size = 32
input_, output = gensym(int, name="input"), gensym(int, name="output")

W = Indexable(torch.randn(input_size, output_size))[input_(), output()]
b = Indexable(torch.randn(output_size))[output()]
X = Indexable(torch.randn(input_size))[input_()]

FullConn(X, W, b, input_)

Indexable(tensor([50.2887, 50.5099, 51.7655, 50.6503, 51.3610, 49.2150, 49.7114, 47.1713,
                  48.9635, 51.5345, 50.5998, 50.2993, 49.1324, 49.7469, 50.2354, 49.6577,
                  48.4640, 50.0672, 50.9392, 51.5744, 48.8719, 52.0908, 52.0701, 53.3827,
                  48.5565, 44.4681, 46.4762, 48.3316, 51.3174, 48.6470, 48.6671, 51.1641]))[output()]

### Recurrent

$$
\begin{aligned}
x^{t} &\in \mathbb{R}^{\mathsf{\vphantom{fg}input}} & t &= 1, \ldots, n \\
W^{\text{h}} &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}\times \mathsf{\vphantom{fg}hidden}^\prime} & |\mathsf{\vphantom{fg}hidden}| &= |\mathsf{\vphantom{fg}hidden}^\prime| \\
W^{\text{i}} &\in \mathbb{R}^{\mathsf{\vphantom{fg}input}\times \mathsf{\vphantom{fg}hidden}^\prime} \\
b &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}^\prime} \\
h^{0} &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}} \\
h^{t} &= \sigma\left( W^{\text{h}} \mathbin{\underset{\substack{\mathsf{\vphantom{fg}hidden}}}{\vphantom{fg}\odot}} h^{t-1} + W^{\text{i}} \mathbin{\underset{\substack{\mathsf{\vphantom{fg}input}}}{\vphantom{fg}\odot}} x^{t} + b \right)_{\mathsf{\vphantom{fg}hidden}^\prime\rightarrow\mathsf{\vphantom{fg}hidden}} & t &= 1, \ldots, n
\end{aligned}
$$

In [108]:
@Operation
def RNN(
    x: torch.Tensor,
    Wh: torch.Tensor,
    Wi: torch.Tensor,
    b: torch.Tensor,
    h: torch.Tensor,
    hidden: Bound,
    layer: Bound,
) -> torch.Tensor:
    return torch.sigmoid(
        reduce([hidden], Wh * h, torch.sum) + reduce([layer], Wi * x, torch.sum) + b
    )

In [109]:
input_size = 100
hidden_size = 32
input_, hidden, hidden2 = gensyms("input", "hidden", "hidden2")

Wh = Indexable(torch.randn(hidden_size, hidden_size))[hidden(), hidden2()]
Wi = Indexable(torch.randn(input_size, hidden_size))[input_(), hidden2()]
b = Indexable(torch.randn(hidden_size))[hidden2()]
h = Indexable(torch.randn(hidden_size))[hidden()]
x = Indexable(torch.randn(input_size))[input_()]

RNN(x, Wh, Wi, b, h, hidden, input_)

Indexable(tensor([1.3253e-06, 9.0270e-01, 9.9990e-01, 6.1722e-06, 1.2837e-03, 1.1857e-14,
                  9.9768e-01, 3.7273e-08, 9.9039e-01, 1.0000e+00, 9.9986e-01, 1.0000e+00,
                  9.7511e-01, 1.2422e-03, 1.0000e+00, 4.8692e-08, 3.0489e-06, 9.9999e-01,
                  4.8624e-10, 2.1622e-01, 6.6186e-03, 9.8496e-01, 8.7746e-01, 9.8309e-01,
                  9.9993e-01, 9.1546e-01, 1.0000e+00, 4.2056e-09, 2.2722e-06, 3.1396e-04,
                  9.9066e-01, 9.9999e-01]))[hidden2()]

### Attention

In [110]:
@Operation
def Softmax(x: torch.Tensor, ax: Bound, ax2: Bound) -> torch.Tensor:
    x = subst(x, {ax: ax2()})
    y = x - reduce([ax2], x, torch.logsumexp)
    return y.exp()

\begin{aligned}
  \text{Attention} \colon \mathbb{R}^{\mathsf{\vphantom{fg}key}} \times \mathbb{R}^{\mathsf{\vphantom{fg}seq}\times\mathsf{\vphantom{fg}key}} \times \mathbb{R}^{\mathsf{\vphantom{fg}seq}\times\mathsf{\vphantom{fg}val}} \times \mathbb{R}^{\mathsf{\vphantom{fg}seq}} &\rightarrow \mathbb{R}^{\mathsf{\vphantom{fg}val}} \\
\text{Attention}(Q, K, V, M) &= \mathop{\underset{\substack{\mathsf{\vphantom{fg}seq}}}{\vphantom{fg}\mathrm{softmax}}} \left( \frac{Q \mathbin{\underset{\substack{\mathsf{\vphantom{fg}key}}}{\vphantom{fg}\odot}} K}{\sqrt{|\mathsf{\vphantom{fg}key}|}} + M \right) \mathbin{\underset{\substack{\mathsf{\vphantom{fg}seq}}}{\vphantom{fg}\odot}} V.
\end{aligned}

In [111]:
@Operation
def Attention(
    Q: torch.Tensor,
    K: torch.Tensor,
    V: torch.Tensor,
    M: torch.Tensor,
    key: Bound,
    seq: Bound,
    seq2: Bound,
) -> torch.Tensor:
    x = reduce([key], Q * K, torch.sum) / sizesof(Q)[key] + M
    return reduce([seq], Softmax(x, seq, seq2) * V, torch.sum)

In [112]:
key_size = 10
val_size = 5
seq_size = 3

key, val, seq, seq2 = gensyms("key", "val", "seq", "seq2")
Q = Indexable(torch.randn(key_size))[key()]
K = Indexable(torch.randn(key_size, seq_size))[key(), seq()]
V = Indexable(torch.randn(seq_size, val_size))[seq(), val()]
M = Indexable(torch.randn(seq_size))[seq()]

Attention(Q, K, V, M, key, seq, seq2)

Indexable(tensor([[0.0127, 0.1222, 0.0344, 0.0174, 0.1486],
                  [0.0799, 0.7717, 0.2169, 0.1098, 0.9383],
                  [0.0152, 0.1466, 0.0412, 0.0209, 0.1783]]))[seq2(), val()]

### Convolution

\begin{aligned}
  \mathop{\underset{\substack{\mathsf{\vphantom{fg}seq}\\ \mathsf{\vphantom{fg}kernel}}}{\vphantom{fg}\mathrm{unroll}}} \colon \mathbb{R}^{\mathsf{\vphantom{fg}seq}[n]} &\rightarrow \mathbb{R}^{\mathsf{\vphantom{fg}seq}[n-|\mathsf{\vphantom{fg}kernel}|+1], \mathsf{\vphantom{fg}kernel}} \\
  \mathop{\underset{\substack{\mathsf{\vphantom{fg}seq}\\ \mathsf{\vphantom{fg}kernel}}}{\vphantom{fg}\mathrm{unroll}}} X &= Y,\ \text{where} \\
  Y_{\mathsf{\vphantom{fg}seq}(i), \mathsf{\vphantom{fg}kernel}(j)} &= X_{\mathsf{\vphantom{fg}seq}(i+j - 1)}.
\end{aligned}

In [130]:
@Operation
def Unroll(
    x: torch.Tensor, seq: Bound, k: int, kernel: Bound, seq2: Bound
) -> torch.Tensor:
    return Indexable(to_tensor(x, [seq]).unfold(0, k, 1))[seq2(), kernel()]

\begin{aligned}
\text{Conv1d} \colon \mathbb{R}^{\mathsf{\vphantom{fg}chans}\times \mathsf{\vphantom{fg}seq}[n]} &\rightarrow \mathbb{R}^{\mathsf{\vphantom{fg}seq}[n^\prime]} \\
\text{Conv1d}(X; W, b) &= W \mathbin{\underset{\substack{\mathsf{\vphantom{fg}chans}\\ \mathsf{\vphantom{fg}kernel}}}{\vphantom{fg}\odot}} \mathop{\underset{\substack{\mathsf{\vphantom{fg}seq}\\ \mathsf{\vphantom{fg}kernel}}}{\vphantom{fg}\mathrm{unroll}}} X + b
\end{aligned}

\begin{aligned}
W &\in \mathbb{R}^{\mathsf{\vphantom{fg}chans}\times \mathsf{\vphantom{fg}kernel}} \\
b &\in \mathbb{R}\\
\end{aligned}

In [131]:
@Operation
def Conv1d(
    X: torch.Tensor,
    W: torch.Tensor,
    b: torch.Tensor,
    chans: Bound,
    k: int,
    kernel: Bound,
    seq: Bound,
    seq2: Bound,
) -> torch.Tensor:
    y = W * Unroll(X, seq, k, kernel, seq2)
    return reduce([chans, kernel], y, torch.sum) + b

In [132]:
chans_size = 3
seq_size = 10
kernel_size = 3

chans, kernel, seq, seq2 = gensyms("chans", "kernel", "seq", "seq2")

X = Indexable(torch.randn(chans_size, seq_size))[chans(), seq()]
W = Indexable(torch.randn(chans_size, kernel_size))[chans(), kernel()]
b = torch.randn(tuple())

Conv1d(X, W, b, chans, 3, kernel, seq, seq2)

Indexable(tensor([-3.1374,  3.5359, -1.3422,  1.3399, -8.3000, -1.6763,  5.6295, -1.6403]))[seq2()]

$$
\begin{aligned}
  \text{Conv2d} \colon \mathbb{R}^{\mathsf{\vphantom{fg}chans}\times \mathsf{\vphantom{fg}height}[h] \times \mathsf{\vphantom{fg}width}[w]}
  &\rightarrow \mathbb{R}^{\mathsf{\vphantom{fg}height}[h2] \times \mathsf{\vphantom{fg}width}[w2]} \\
  \text{Conv2d}(X; W, b) &= W \mathbin{\underset{\substack{\mathsf{\vphantom{fg}chans}\\ \mathsf{\vphantom{fg}kh}, \mathsf{\vphantom{fg}kw}}}{\vphantom{fg}\odot}} \mathop{\underset{\substack{\mathsf{\vphantom{fg}height}\\ \mathsf{\vphantom{fg}kh}}}{\vphantom{fg}\mathrm{unroll}}} \mathop{\underset{\substack{\mathsf{\vphantom{fg}width}\\\mathsf{\vphantom{fg}kw}}}{\vphantom{fg}\mathrm{unroll}}} X + b\end{aligned}
$$

$$
\begin{aligned}
W &\in \mathbb{R}^{\mathsf{\vphantom{fg}chans}\times \mathsf{\vphantom{fg}kh}\times \mathsf{\vphantom{fg}kw}} \\
b &\in \mathbb{R}.
\end{aligned}
$$

In [116]:
@Operation
def Conv2d(
    X: torch.Tensor,
    W: torch.Tensor,
    b: torch.Tensor,
    chans: Bound,
    kh_size: int,
    kh: Bound,
    height: Bound,
    height2: Bound,
    kw: Bound,
    width: Bound,
    width2: Bound,
) -> torch.Tensor:
    y = W * Unroll(Unroll(X, width, kw_size, kw, width2), height, kh_size, kh, height2)
    return reduce([chans, kh, kw], y, torch.sum) + b

In [117]:
chans_size = 3
kh_size = 3
kw_size = 4
height_size = 10
width_size = 8

chans, kh, kw, height, width, height2, width2 = gensyms(
    "chans", "kh", "kw", "height", "width", "height2", "width2"
)

X = Indexable(torch.randn(chans_size, height_size, width_size))[
    chans(), height(), width()
]
W = Indexable(torch.randn(chans_size, kh_size, kw_size))[chans(), kh(), kw()]
b = torch.randn(tuple())

Conv2d(X, W, b, chans, kh_size, kh, height, height2, kw, width, width2)

Indexable(tensor([[  8.2551,  -3.6813, -10.3426,  -2.9706,  -2.5484,  -3.7450,   0.2684,
                     8.0988],
                  [-14.0129,   6.7571,  -0.4884,  -5.4678, -12.0890,  -3.1195,  -5.7605,
                   -13.3160],
                  [ -0.6627,  -0.7033,  -6.0278,  -2.7052,  -2.6699,   5.0469,  -2.4248,
                    -8.7640],
                  [  2.4971,   6.5268,   4.0790,  -0.0584,   1.7761, -10.4241,   3.2864,
                    -5.5535],
                  [ -2.9754, -11.3655, -16.1200,  -5.5068,   8.5963,  12.9700,   8.1375,
                     0.3483]]))[width2(), height2()]

### Max Pooling

$$
\begin{aligned}
  \mathop{\underset{\substack{\mathsf{\vphantom{fg}seq},\mathsf{\vphantom{fg}kernel}}}{\vphantom{fg}\mathrm{pool}}} \colon \mathbb{R}^{\mathsf{\vphantom{fg}seq}[n]} &\rightarrow \mathbb{R}^{\mathsf{\vphantom{fg}seq}[n/|\mathsf{\vphantom{fg}kernel}|],\mathsf{\vphantom{fg}kernel}} \\
  \mathop{\underset{\substack{\mathsf{\vphantom{fg}seq},\mathsf{\vphantom{fg}kernel}}}{\vphantom{fg}\mathrm{pool}}} X &= Y,\ \text{where} \\
  Y_{\mathsf{\vphantom{fg}seq}(i), \mathsf{\vphantom{fg}kernel}(j)} &= X_{\mathsf{\vphantom{fg}seq}((i-1) \cdot |\mathsf{\vphantom{fg}kernel}| + j)}.
\end{aligned}
$$

In [118]:
@Operation
def Pool(
    x: torch.Tensor, seq: Bound, k: int, kernel: Bound, seq2: Bound
) -> torch.Tensor:
    xp = to_tensor(x, [seq])
    return Indexable(xp.reshape((xp.shape[0] // k, k) + xp.shape[1:]))[seq2(), kernel()]

In [119]:
seq_size = 10
seq, seq2, kernel = gensyms("seq", "seq2", "kernel")

X = Indexable(torch.randn(seq_size))[seq()]
Y = Pool(X, seq, 2, kernel, seq2)
Y

Indexable(tensor([[ 1.3453,  0.4969],
                  [-0.2328,  1.1193],
                  [ 0.8813,  1.4646],
                  [ 0.3213, -0.0067],
                  [ 1.6483,  2.6870]]))[seq2(), kernel()]

$$
\begin{aligned}
\text{MaxPool1d}_{k} \colon \mathbb{R}^{\mathsf{\vphantom{fg}seq}[n]} &\rightarrow \mathbb{R}^{\mathsf{\vphantom{fg}seq}[n/k]} \\
\text{MaxPool1d}_{k}(X) &= \mathop{\underset{\substack{\mathsf{\vphantom{fg}kernel}}}{\vphantom{fg}\mathrm{max}}} \mathop{\underset{\substack{\mathsf{\vphantom{fg}seq},\mathsf{\vphantom{fg}kernel}}}{\vphantom{fg}\mathrm{pool}}} X \\
|\mathsf{\vphantom{fg}kernel}| &= k \\
\text{MaxPool2d}_{kh,kw} \colon \mathbb{R}^{\mathsf{\vphantom{fg}height}[h] \times \mathsf{\vphantom{fg}width}[w]} &\rightarrow \mathbb{R}^{\mathsf{\vphantom{fg}height}[h/kh] \times \mathsf{\vphantom{fg}width}[w/kw]} \\
\text{MaxPool2d}_{kh,kw}(X) &= \mathop{\underset{\substack{\mathsf{\vphantom{fg}kh},\mathsf{\vphantom{fg}kw}}}{\vphantom{fg}\mathrm{max}}} \mathop{\underset{\substack{\mathsf{\vphantom{fg}height},\mathsf{\vphantom{fg}kh}}}{\vphantom{fg}\mathrm{pool}}} \mathop{\underset{\substack{\mathsf{\vphantom{fg}width},\mathsf{\vphantom{fg}kw}}}{\vphantom{fg}\mathrm{pool}}} X \\
|\mathsf{\vphantom{fg}kh}| &= kh \\
|\mathsf{\vphantom{fg}kw}| &= kw.
\end{aligned}
$$

In [120]:
@Operation
def MaxPool1d(
    X: torch.Tensor, seq: Bound, k: int, kernel: Bound, seq2: Bound
) -> torch.Tensor:
    return reduce([kernel], Pool(X, seq, k, kernel, seq2), torch.max)

In [121]:
seq_size = 10

seq, seq2, kernel = gensyms("seq", "seq2", "kernel")

X = Indexable(torch.randn(seq_size))[seq()]
MaxPool1d(X, seq, 2, kernel, seq2)

Indexable(tensor([[-0.3350],
                  [-0.3891],
                  [-1.0657],
                  [-0.2448],
                  [ 0.4492]]))[seq2(), slice(None, None, None)]

In [122]:
@Operation
def MaxPool2d(
    X: torch.Tensor,
    height: Bound,
    kh_size: int,
    kh: Bound,
    height2: Bound,
    width: Bound,
    kw_size: int,
    kw: Bound,
    width2: Bound,
) -> torch.Tensor:
    y = Pool(Pool(X, height, kh_size, kh, height2), width, kw_size, kw, width2)
    return reduce([kh, kw], y, torch.max)

In [123]:
width_size = 9
height_size = 4

width, width2, height, height2, kw, kh = gensyms(
    "width", "width2", "height", "height2", "kw", "kh"
)

X = Indexable(torch.randn(width_size, height_size))[width(), height()]
MaxPool2d(X, height, 2, kh, height2, width, 3, kw, width2)

Indexable(tensor([[[0.7018],
                   [1.3316],
                   [1.5385]],
          
                  [[0.8195],
                   [1.7751],
                   [0.7409]]]))[height2(), width2(), slice(None, None, None)]

### Normalization Layers

$$
\begin{aligned}
  \mathop{\underset{\substack{\mathsf{\vphantom{fg}ax}}}{\vphantom{fg}\mathrm{standardize}}} \colon \mathbb{R}^{\mathsf{\vphantom{fg}ax}} &\rightarrow \mathbb{R}^{\mathsf{\vphantom{fg}ax}} \\
  \mathop{\underset{\substack{\mathsf{\vphantom{fg}ax}}}{\vphantom{fg}\mathrm{standardize}}}(X) &= \frac{X - \mathop{\underset{\substack{\mathsf{\vphantom{fg}ax}}}{\vphantom{fg}\mathrm{mean}}}(X)}{\sqrt{\mathop{\underset{\substack{\mathsf{\vphantom{fg}ax}}}{\vphantom{fg}\mathrm{var}}}(X) + \epsilon}}
\end{aligned}
$$

In [124]:
@Operation
def Mean(X: torch.Tensor, ax: Bound) -> torch.Tensor:
    return reduce([ax], X, torch.sum) / sizesof(X)[ax]


@Operation
def Mean2(X: torch.Tensor, ax: Bound, ax2: Bound) -> torch.Tensor:
    sizes = sizesof(X)
    return reduce([ax, ax2], X, torch.sum) / (sizes[ax] * sizes[ax2])


@Operation
def Variance(X: torch.Tensor, ax: Bound) -> torch.Tensor:
    return Mean((X - Mean(X, ax)) ** 2, ax)


@Operation
def Variance2(X: torch.Tensor, ax: Bound, ax2: Bound) -> torch.Tensor:
    return Mean2((X - Mean2(X, ax, ax2)) ** 2, ax, ax2)


@Operation
def Standardize(X: torch.Tensor, ax: Bound, new_ax: Bound) -> torch.Tensor:
    y = subst(X, {ax: new_ax()})
    return (y - Mean(X, ax)) / (Variance(X, ax) + torch.finfo(X.dtype).eps).sqrt()


@Operation
def Standardize2(
    X: torch.Tensor,
    ax: Bound,
    ax2: Bound,
    new_ax: Bound,
    new_ax2: Bound,
) -> torch.Tensor:
    y = subst(X, {ax: new_ax(), ax2: new_ax2()})
    return (y - Mean2(X, ax, ax2)) / (
        Variance2(X, ax, ax2) + torch.finfo(X.dtype).eps
    ).sqrt()

$$
\begin{aligned}
\text{BatchNorm}(X; \gamma, \beta) &= \mathop{\underset{\substack{\mathsf{\vphantom{fg}batch},\mathsf{\vphantom{fg}layer}}}{\vphantom{fg}\mathrm{standardize}}}(X) \mathbin{\underset{\substack{}}{\vphantom{fg}\odot}} \gamma + \beta & \gamma, \beta &\in \mathbb{R}^{\mathsf{\vphantom{fg}chans}} \\
\text{InstanceNorm}(X; \gamma, \beta) &= \mathop{\underset{\substack{\mathsf{\vphantom{fg}layer}}}{\vphantom{fg}\mathrm{standardize}}}(X) \mathbin{\underset{\substack{}}{\vphantom{fg}\odot}} \gamma + \beta & \gamma, \beta &\in \mathbb{R}^{\mathsf{\vphantom{fg}chans}} \\
\text{LayerNorm}(X; \gamma, \beta) &= \mathop{\underset{\substack{\mathsf{\vphantom{fg}layer},\mathsf{\vphantom{fg}chans}}}{\vphantom{fg}\mathrm{standardize}}}(X) \mathbin{\underset{\substack{}}{\vphantom{fg}\odot}} \gamma + \beta & \gamma, \beta &\in \mathbb{R}^{\mathsf{\vphantom{fg}chans},\mathsf{\vphantom{fg}layer}}
\end{aligned}
$$

In [125]:
@Operation
def BatchNorm(
    X: torch.Tensor,
    gamma: torch.Tensor,
    beta: torch.Tensor,
    batch: Bound,
    layer: Bound,
    batch2: Bound,
    layer2: Bound,
) -> torch.Tensor:
    return Standardize2(X, batch, layer, batch2, layer2) * gamma + beta


@Operation
def InstanceNorm(
    X: torch.Tensor,
    gamma: torch.Tensor,
    beta: torch.Tensor,
    layer: Bound,
    layer2: Bound,
) -> torch.Tensor:
    return Standardize(X, layer, layer2) * gamma + beta


# same as BatchNorm
@Operation
def LayerNorm(
    X: torch.Tensor,
    gamma: torch.Tensor,
    beta: torch.Tensor,
    chans: Bound,
    layer: Bound,
    chans2: Bound,
    layer2: Bound,
) -> torch.Tensor:
    return Standardize2(X, chans, layer, chans2, layer2) * gamma + beta

In [126]:
batch_size, chans_size, layer_size = 4, 3, 5
batch, batch2, chans, layer, layer2 = gensyms(
    "batch", "batch2", "chans", "layer", "layer2"
)

x = Indexable(torch.randn(batch_size, chans_size, layer_size))[
    batch(), chans(), layer()
]
g = Indexable(torch.randn(chans_size))[chans()]
b = Indexable(torch.randn(chans_size))[chans()]

BatchNorm(x, g, b, batch, layer, batch2, layer2)

Indexable(tensor([[[-1.3660e+00, -1.2092e+00, -1.1914e+00, -1.1869e+00, -3.2031e+00],
                   [ 4.4035e-01,  2.2923e-01, -2.9452e-01,  4.6736e-01, -5.2524e-04],
                   [-2.7548e-01, -2.3009e-01,  9.1457e-01,  3.5616e-02, -8.6602e-01]],
          
                  [[-1.5035e+00, -4.4308e+00, -2.0138e+00, -1.3974e+00, -2.1422e+00],
                   [-3.4515e-02,  4.0153e-01, -4.4831e-02, -4.8727e-03,  2.2006e-01],
                   [ 2.4698e-01, -1.9679e-01, -1.7004e-01, -2.1854e-01, -8.8352e-01]],
          
                  [[-1.3800e+00, -5.0026e+00, -9.1291e-01, -1.0744e+00, -4.3030e-01],
                   [ 2.3899e-01,  1.3271e-01,  2.4747e-01,  6.7415e-01,  1.9799e-01],
                   [ 4.6958e-01,  4.7553e-01,  1.4773e+00,  4.6914e-01,  5.6096e-01]],
          
                  [[-3.1203e+00, -2.3236e+00, -9.2935e-01, -3.7449e+00, -5.7545e-01],
                   [ 2.4362e-01,  6.4439e-01, -2.1590e-01,  5.2150e-01,  3.0277e-01],
                  

$$
\begin{aligned}
\text{GroupNorm}_k(X; \gamma, \beta) &= \left[ \mathop{\underset{\substack{\mathsf{\vphantom{fg}kernel},\mathsf{\vphantom{fg}layer}}}{\vphantom{fg}\mathrm{standardize}}} \mathop{\underset{\substack{\mathsf{\vphantom{fg}chans}, \mathsf{\vphantom{fg}kernel}}}{\vphantom{fg}\mathrm{pool}}} X \right]_{(\mathsf{\vphantom{fg}chans},\mathsf{\vphantom{fg}kernel})\rightarrow \mathsf{\vphantom{fg}chans}} \mathbin{\underset{\substack{}}{\vphantom{fg}\odot}} \gamma + \beta \\
\end{aligned}
$$

$$
\begin{aligned}
|\mathsf{\vphantom{fg}kernel}| &= k\\
\gamma, \beta &\in \mathbb{R}^{\mathsf{\vphantom{fg}chans}}.
\end{aligned}
$$

### Transformer

$$
\begin{aligned}
  I &\in \{0, 1\}^{\mathsf{\vphantom{fg}seq}\times \mathsf{\vphantom{fg}vocab}} & \sum\limits_{\substack{\mathsf{\vphantom{fg}vocab}}} I &= 1 \\
  W &= (E \mathbin{\underset{\substack{\mathsf{\vphantom{fg}vocab}}}{\vphantom{fg}\odot}} I)\sqrt{|\mathsf{\vphantom{fg}layer}|} & E &\in \mathbb{R}^{\mathsf{\vphantom{fg}vocab}\times \mathsf{\vphantom{fg}layer}} \\
  P &\in \mathbb{R}^{\mathsf{\vphantom{fg}seq}\times \mathsf{\vphantom{fg}layer}} \\
  P_{\mathsf{\vphantom{fg}seq}(p), \mathsf{\vphantom{fg}layer}(i)} &= \begin{cases}
    \sin((p-1) / 10000^{(i-1) / |\mathsf{\vphantom{fg}layer}|}) & \text{$i$ odd} \\ 
    \cos((p-1) / 10000^{(i-2) / |\mathsf{\vphantom{fg}layer}|}) & \text{$i$ even.}
  \end{cases}
\end{aligned}
$$

$$
\begin{aligned}
X^0 &= W+P \\
T^1 &= \text{LayerNorm}^1(\text{SelfAtt}^1(X^0)) + X^0\\
X^1 &= \text{LayerNorm}^{1^\prime}(\text{FFN}^1(T^1)) + T^1\\
&\vdotswithin{=} \\
T^{L} &= \text{LayerNorm}^L(\text{SelfAtt}^L(X^{L-1})) + X^{L-1}\\
X^{L} &= \text{LayerNorm}^{L^\prime}(\text{FFN}^L(T^L)) + T^L\\
O &= \mathop{\underset{\substack{\mathsf{\vphantom{fg}vocab}}}{\vphantom{fg}\mathrm{softmax}}}(E \mathbin{\underset{\substack{\mathsf{\vphantom{fg}layer}}}{\vphantom{fg}\odot}} X^L)
\end{aligned}
$$

$$
\begin{aligned}
  \text{LayerNorm}^l \colon \mathbb{R}^{\mathsf{\vphantom{fg}layer}} &\rightarrow \mathbb{R}^{\mathsf{\vphantom{fg}layer}} \\
  \text{LayerNorm}^l(X) &= \mathop{\underset{\substack{\mathsf{\vphantom{fg}layer}}}{\vphantom{fg}\mathrm{XNorm}}}(X; \beta^l, \gamma^l).
\end{aligned}
$$

$$
\begin{aligned}
  \text{SelfAtt}^l \colon \mathbb{R}^{\mathsf{\vphantom{fg}seq}\times \mathsf{\vphantom{fg}layer}} &\rightarrow \mathbb{R}^{\mathsf{\vphantom{fg}seq}\times \mathsf{\vphantom{fg}layer}} \\
  \text{SelfAtt}^l(X) &= Y
\end{aligned}
$$

$$
\begin{aligned}
  |\mathsf{\vphantom{fg}seq}| &= |\mathsf{\vphantom{fg}seq2}| \\
  |\mathsf{\vphantom{fg}key}| = |\mathsf{\vphantom{fg}val}| &= |\mathsf{\vphantom{fg}layer}|/|\mathsf{\vphantom{fg}heads}| \\
  Q &= W^{l,Q} \mathbin{\underset{\substack{\mathsf{\vphantom{fg}layer}}}{\vphantom{fg}\odot}} X_{\mathsf{\vphantom{fg}seq}\rightarrow\mathsf{\vphantom{fg}seq2}} & W^{l,Q} &\in \mathbb{R}^{\mathsf{\vphantom{fg}heads}\times \mathsf{\vphantom{fg}layer}\times \mathsf{\vphantom{fg}key}} \\
  K &= W^{l,K} \mathbin{\underset{\substack{\mathsf{\vphantom{fg}layer}}}{\vphantom{fg}\odot}} X & W^{l,K} &\in \mathbb{R}^{\mathsf{\vphantom{fg}heads}\times \mathsf{\vphantom{fg}layer}\times \mathsf{\vphantom{fg}key}} \\
  V &= W^{l,V} \mathbin{\underset{\substack{\mathsf{\vphantom{fg}layer}}}{\vphantom{fg}\odot}} X & W^{l,V} &\in \mathbb{R}^{\mathsf{\vphantom{fg}heads}\times \mathsf{\vphantom{fg}layer}\times \mathsf{\vphantom{fg}val}} \\
  M & \in \mathbb{R}^{\mathsf{\vphantom{fg}seq}\times \mathsf{\vphantom{fg}seq2}} \\
  M_{\mathsf{\vphantom{fg}seq}(i), \mathsf{\vphantom{fg}seq2}(j)} &= \begin{cases}
    0 & i \leq j\\
    -\infty & \text{otherwise}
  \end{cases} \\
  Y &= W^{l,O} \mathbin{\underset{\substack{\mathsf{\vphantom{fg}heads}\\ \mathsf{\vphantom{fg}val}}}{\vphantom{fg}\odot}} \text{Attention}(Q, K, V, M)_{\mathsf{\vphantom{fg}seq2}\rightarrow\mathsf{\vphantom{fg}seq}} & W^{l,O} &\in \mathbb{R}^{\mathsf{\vphantom{fg}heads}\times \mathsf{\vphantom{fg}val}\times \mathsf{\vphantom{fg}layer}}
\end{aligned}
$$

$$
\begin{aligned}
  \text{FFN}^l \colon \mathbb{R}^{\mathsf{\vphantom{fg}layer}} &\rightarrow \mathbb{R}^{\mathsf{\vphantom{fg}layer}} \\
  \text{FFN}^l(X) &= X^2
\end{aligned}
$$

$$
\begin{aligned}
  X^1 &= \text{relu}(W^{l,1} \mathbin{\underset{\substack{\mathsf{\vphantom{fg}layer}}}{\vphantom{fg}\odot}} X + b^{l,1}) & W^{l,1} &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}\times \mathsf{\vphantom{fg}layer}} & b^{l,1} &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}} \\
  X^2 &= \text{relu}(W^{l,2} \mathbin{\underset{\substack{\mathsf{\vphantom{fg}hidden}}}{\vphantom{fg}\odot}} X^1 + b^{l,2}) & W^{l,2} &\in \mathbb{R}^{\mathsf{\vphantom{fg}layer}\times \mathsf{\vphantom{fg}hidden}} & b^{l,2} &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}}.
\end{aligned}
$$

### LeNet

$$
\begin{aligned}
X^0 &\in \mathbb{R}^{\mathsf{\vphantom{fg}batch}\times \mathsf{\vphantom{fg}chans}[c_0] \times \mathsf{\vphantom{fg}height}\times \mathsf{\vphantom{fg}width}} \\
T^1 &= \text{relu}(\text{Conv}^1(X^0)) \\
X^1 &= \text{MaxPool}^1(T^1) \\
T^2 &= \text{relu}(\text{Conv}^2(X^1)) \\
X^2 &= \text{MaxPool}^2(T^2)_{(\mathsf{\vphantom{fg}height},\mathsf{\vphantom{fg}width},\mathsf{\vphantom{fg}chans})\rightarrow\mathsf{\vphantom{fg}layer}} \\
X^3 &= \text{relu}(W^3 \mathbin{\underset{\substack{\mathsf{\vphantom{fg}layer}}}{\vphantom{fg}\odot}} X^2 + b^3) & W^3 &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}\times \mathsf{\vphantom{fg}layer}} & b^3 &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}} \\
O &= \mathop{\underset{\substack{\mathsf{\vphantom{fg}classes}}}{\vphantom{fg}\mathrm{softmax}}} (W^4 \mathbin{\underset{\substack{\mathsf{\vphantom{fg}hidden}}}{\vphantom{fg}\odot}} X^3 + b^4) & W^4 &\in \mathbb{R}^{\mathsf{\vphantom{fg}classes}\times \mathsf{\vphantom{fg}hidden}} & b^4 &\in \mathbb{R}^{\mathsf{\vphantom{fg}classes}}\end{aligned}
$$

$$
\begin{aligned}
X^2 &= \text{MaxPool}^2(T^2) \\
X^3 &= \text{relu}(W^3 \mathbin{\underset{\substack{\mathsf{\vphantom{fg}height}\\ \mathsf{\vphantom{fg}width}\\ \mathsf{\vphantom{fg}chans}}}{\vphantom{fg}\odot}} X^2 + b^3) & W^3 &\in \mathbb{R}^{\mathsf{\vphantom{fg}hidden}\times \mathsf{\vphantom{fg}height}\times \mathsf{\vphantom{fg}width}\times \mathsf{\vphantom{fg}chans}}.
\end{aligned}
$$

$$
\begin{aligned}
\text{Conv}^l(X) &= \text{Conv2d}(X; W^l, b^l)_{\mathsf{\vphantom{fg}chans2}\rightarrow\mathsf{\vphantom{fg}chans}}
\end{aligned}
$$

$$
\begin{aligned}
W^l & \in \mathbb{R}^{\mathsf{\vphantom{fg}chans2}[c_l] \times \mathsf{\vphantom{fg}chans}[c_{l-1}] \times \mathsf{\vphantom{fg}kh}[kh_l] \times \mathsf{\vphantom{fg}kw}[kw_l]} \\
b^l &\in \mathbb{R}^{\mathsf{\vphantom{fg}chans2}[c_l]}
\end{aligned}
$$

$$
\begin{aligned}
\text{MaxPool}^l(X) &amp;= \text{MaxPool2d}_{ph^l,ph^l}(X).
\end{aligned}
$$

In [127]:
@Operation
def Relu(X: torch.Tensor) -> torch.Tensor:
    return torch.maximum(X, torch.tensor(0))

In [128]:
Relu(x)

Indexable(tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.4903],
                   [1.0049, 0.3939, 0.0000, 1.0830, 0.0000],
                   [0.0466, 0.0044, 0.0000, 0.0000, 0.5950]],
          
                  [[0.0000, 1.1864, 0.0000, 0.0000, 0.0000],
                   [0.0000, 0.8925, 0.0000, 0.0000, 0.3674],
                   [0.0000, 0.0000, 0.0000, 0.0000, 0.6112]],
          
                  [[0.0000, 1.5107, 0.0000, 0.0000, 0.0000],
                   [0.4221, 0.1146, 0.4467, 1.6815, 0.3035],
                   [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],
          
                  [[0.4434, 0.0000, 0.0000, 0.7975, 0.0000],
                   [0.4355, 1.5954, 0.0000, 1.2397, 0.6067],
                   [0.0000, 0.0918, 1.2267, 0.3206, 0.8082]]]))[batch(), chans(), layer()]

In [129]:
(
    chans_size,
    kh_size,
    kw_size,
    hidden_size,
    height_size,
    width_size,
    classes_size,
    batch_size,
) = (3, 3, 4, 3, 14, 15, 5, 4)
(
    chans,
    chans2,
    kh,
    kw,
    height,
    height2,
    height3,
    width,
    width2,
    width3,
    hidden,
    classes,
    classes2,
    batch,
) = gensyms(
    "chans",
    "chans2",
    "kh",
    "kw",
    "height",
    "height2",
    "height3",
    "width",
    "width2",
    "width3",
    "hidden",
    "classes",
    "classes2",
    "batch",
)

W1 = Indexable(torch.randn(chans_size, kh_size, kw_size, chans_size))[
    chans(), kh(), kw(), chans2()
]
b1 = Indexable(torch.randn(chans_size))[chans2()]
W3 = Indexable(torch.randn(hidden_size, 4, 4, chans_size))[
    hidden(), height3(), width3(), chans2()
]
b3 = Indexable(torch.randn(hidden_size))[hidden()]
W4 = Indexable(torch.randn(hidden_size, classes_size))[hidden(), classes()]
b4 = Indexable(torch.randn(classes_size))[classes()]
X0 = Indexable(torch.randn(batch_size, chans_size, height_size, width_size))[
    batch(), chans(), height(), width()
]

T1 = Relu(Conv2d(X0, W1, b1, chans, kh_size, kh, height, height2, kw, width, width2))
X1 = MaxPool2d(T1, height2, 3, kh, height3, width2, 3, kw, width3)
X3 = reduce([height3, width3, chans2], W3 * X1, torch.sum) + b3
O = Softmax(reduce([hidden], W4 * X3, torch.sum) + b4, classes, classes2)
O

Indexable(tensor([[[4.0487e-28],
                   [1.5080e-13],
                   [0.0000e+00],
                   [1.0000e+00]],
          
                  [[1.0000e+00],
                   [1.0000e+00],
                   [1.0000e+00],
                   [3.9901e-34]],
          
                  [[2.7085e-17],
                   [5.3837e-24],
                   [1.3339e-36],
                   [0.0000e+00]],
          
                  [[2.7742e-33],
                   [5.2604e-33],
                   [0.0000e+00],
                   [0.0000e+00]],
          
                  [[1.6050e-17],
                   [3.8066e-11],
                   [0.0000e+00],
                   [1.6255e-43]]]))[classes2(), batch(), slice(None, None, None)]